In [167]:
import pandas as pd


# STEP 1: Load the datasets
### We read the financial dataset (box office + budget) and IMDb dataset (ratings + votes + language)
### These files are collected from real web sources and stored locally.

In [168]:
financial= pd.read_csv('data/movies_financial.csv')
imdb= pd.read_csv('data/imdb_data.csv')

In [169]:
financial.head()

,Release Date,Title,Budget,Domestic Gross,Worldwide Gross
0,"Dec 16, 2015",Star Wars Ep. VII: The Force Awakens,"$533,200,000","$936,662,225","$2,056,046,835"
1,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,717,503,922"
2,"Dec 9, 2022",Avatar: The Way of Water,"$400,000,000","$688,809,501","$2,322,902,023"
3,"May 17, 2025",Mission: Impossible—The Final Reckoning,"$400,000,000","$197,413,515","$591,353,074"
4,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,071,802","$1,045,713,802"


In [170]:
imdb.head()

,Title,Year,Rating,Vote Count,Language
0,The Shawshank Redemption,1994.0,8.713,29336,en
1,The Godfather,1972.0,8.685,22140,en
2,The Godfather Part II,1974.0,8.571,13384,en
3,Schindler's List,1993.0,8.566,16896,en
4,12 Angry Men,1957.0,8.500,9579,en


# STEP 2: Normalize movie titles for matching
### We create a helper column "Title_clean" which:
###  - Converts titles to lowercase
###  - Removes extra spaces before/after
### This helps matching movies even if formatting differs between files.

In [171]:
financial["Title_clean"] = financial["Title"].str.lower().str.strip()
imdb["Title_clean"] = imdb["Title"].str.lower().str.strip()


# STEP 3: Convert Release Date to datetime format
### `errors="coerce"` will convert invalid or unknown dates to NaT instead of crashing.




In [172]:
financial["Release Date"] = pd.to_datetime(
    financial["Release Date"],
    errors="coerce",
    
)

# STEP 4: Extract the year from Release Date
### This will be used as part of the merge key, because many movies share the same title
### but are different films from different years.

In [173]:
financial["Year"] = financial["Release Date"].dt.year

# STEP 5: Merge financial + IMDb using the cleaned title and year
### We use `how="inner"` meaning:
###             Only movies that exist in BOTH datasets will be kept.
### This avoids merging unrelated movies.

In [174]:
merged = pd.merge(
    financial, imdb, how="inner", on=["Title_clean", "Year"]
)


# STEP 6: Fix duplicate Title columns created by merge
### Pandas adds _x and _y when two files have the same column name.
### We:
###  - Drop "Title_y" (IMDb original Title)
###  - Rename "Title_x" back to "Title" (financial original Title)


In [175]:
merged = merged.drop(columns=["Title_y"], errors="ignore")
merged = merged.rename(columns={"Title_x": "Title"})

# STEP 7: Remove the helper column "Title_clean"
### We no longer need it after merging.

In [176]:
merged = merged.drop(columns=["Title_clean"], errors="ignore")


# STEP 8: add season cloumn

In [177]:
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    elif month in [9, 10, 11]:
        return "Fall"
    else:
        return None

merged["Season"] = merged["Release Date"].dt.month.apply(get_season)


# STEP 8: Save the merged dataset for later use
### This file now contains:
###  Release Date, Title, Budget, Domestic Gross, Worldwide Gross, Rating, Vote Count, Language, Year


In [178]:
merged.to_csv(r"data\movies_dataset.csv", index=False)

# STEP 9: Reload the cleaned merged dataset to continue integration

In [179]:
df_mine = pd.read_csv(r"data\movies_dataset.csv")

# STEP 10: Load Rotten Tomatoes dataset
### This dataset contains critic and audience scores, which we want to integrate too.

In [180]:
df_rt = pd.read_csv(r"data\rotten_tomatoes_movies.csv")

# STEP 11: Keep only the useful columns from Rotten Tomatoes
### We ignore unnecessary columns to keep the dataset clean and analysis-ready.

In [181]:
df_rt_clean = df_rt[['movie_title', 'original_release_date', 'tomatometer_rating', 'audience_rating','runtime','genres','content_rating']].copy()


# STEP 12: Normalize titles again for RT dataset
### This ensures RT titles follow the same format as the other datasets.

In [182]:
df_rt_clean["Title_clean"] = df_rt_clean["movie_title"].str.lower().str.strip()

# STEP 13: Extract the release year from Rotten Tomatoes release date
### This will allow us to match movies across all 3 sources.

In [183]:
df_rt_clean['Year'] = pd.to_datetime(df_rt_clean['original_release_date'], errors="coerce").dt.year


# STEP 14: Rename RT columns to clearer names
### This makes them easier to use in analysis and avoids confusion.

In [184]:
df_rt_clean = df_rt_clean.rename(columns={
    'movie_title': 'Title',
    'tomatometer_rating': 'Critic_Score',
    'audience_rating': 'Audience_Score'
})

# STEP 15: Remove the original helper column we used for cleaning

In [185]:
df_rt_clean = df_rt_clean.drop(columns=["movie_title"], errors="ignore")
df_rt_clean = df_rt_clean.drop(columns=["Title_clean"], errors="ignore")
df_rt_clean = df_rt_clean.drop(columns=["original_release_date"], errors="ignore")





# STEP 16: Final merge → integrate financial+IMDb dataset with Rotten Tomatoes
### Again using `inner` to keep only movies found in ALL datasets.
### This final dataset is now fully integrated and ready for prediction analysis.

In [186]:
final = pd.merge(df_mine, df_rt_clean, on=['Title', 'Year'], how='inner')

# STEP 17: Save the fully integrated dataset
### This is the dataset you will use in your project for EDA and modeling.

In [187]:
final.to_csv(r"data\movies_project_final_dataset.csv", index=False)


# STEP 18: Display final dataset size and structure
### This confirms merge success and shows final columns + data types.

In [188]:

print("Final dataset shape:", final.shape)
final.info()

Final dataset shape: (1995, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1995 entries, 0 to 1994
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Release Date     1995 non-null   object 
 1   Title            1995 non-null   object 
 2   Budget           1995 non-null   object 
 3   Domestic Gross   1995 non-null   object 
 4   Worldwide Gross  1995 non-null   object 
 5   Year             1995 non-null   float64
 6   Rating           1995 non-null   float64
 7   Vote Count       1995 non-null   int64  
 8   Language         1995 non-null   object 
 9   Season           1995 non-null   object 
 10  Critic_Score     1992 non-null   float64
 11  Audience_Score   1992 non-null   float64
 12  runtime          1983 non-null   float64
 13  genres           1995 non-null   object 
 14  content_rating   1995 non-null   object 
dtypes: float64(5), int64(1), object(9)
memory usage: 233.9+ KB
